You might need to install this on your system:

apt-get install python3-opencv git

In [1]:
import os
#"""
# ! rm -r k
if not os.path.isdir('k'):
 !git clone -b development12 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
#"""
!cd k && pip install .

Already up to date.
Processing /tf/k
  Created wheel for cai: filename=cai-0.1.4-py3-none-any.whl size=56793 sha256=56d02a5df837cbb0bda2e775ea0814b70d10e0fd7a3f81b42e42d847f4a0395e
  Stored in directory: /tmp/pip-ephem-wheel-cache-bllicx9a/wheels/f0/08/19/56f64e8c8cc45b0390e5e7e2f634c4c1aa0212065044fb6442
Successfully built cai
  Attempting uninstall: cai
    Found existing installation: cai 0.1.4
    Uninstalling cai-0.1.4:
      Successfully uninstalled cai-0.1.4
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import cai.layers
import cai.datasets
import cai.models
import cai.mobilenet
import numpy as np
from tensorflow import keras
from tensorflow.keras import mixed_precision
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
from sklearn.metrics import classification_report

Tensorflow version: 2.6.0
Keras version: 2.6.0


In [3]:
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(
      model.call,
      input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

  graph_info = profile(forward_pass.get_concrete_function().graph,
                          options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  return flops

In [4]:
# mixed_precision.set_global_policy('mixed_float16')

# Create folder structure with CIFAR-10 as png files.

In [5]:
verbose=True
root_folder = 'cifar10-as-png'
data_dir = root_folder + '/train';
test_dir = root_folder + '/test';

if not os.path.isdir(root_folder):
  x_train, y_train, x_test, y_test = cai.datasets.load_dataset(tf.keras.datasets.cifar10, verbose=verbose, lab=False, bipolar=False)
  os.mkdir(root_folder)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_train)*255, y_train, dest_folder_name=data_dir)
  cai.datasets.save_dataset_as_png(cai.datasets.fix_bad_tfkeras_channel_order(x_test)*255, y_test, dest_folder_name=test_dir)

In [6]:
num_classes = 10
batch_size = 64
epochs = 50
target_size_x = 224
target_size_y = 224
seed = 12

In [7]:
train_datagen = cai.util.create_image_generator(validation_split=0.1, rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.0, rescale=1./255)
test_datagen = cai.util.create_image_generator_no_augmentation(rescale=1./255)
cpus_num = max([multiprocessing.cpu_count(), 8])

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [28]:
def work_on_kmobilenet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [26]: #
      basefilename = 'JP35B18-mobilenet-CIFAR10-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == -1:
        model = cai.mobilenet.MobileNet(
          include_top=True,
          weights=None,
          input_tensor=None,
          input_shape=input_shape,
          pooling=None,
          classes=num_classes)
      else:
        model = cai.mobilenet.kMobileNet(
          include_top=True,
          weights=None,
          input_shape=input_shape,
          pooling=None,
          classes=num_classes,
          kType=kType)
        
      #opt = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model):
        model.summary()
        print('model flops:',get_flops(model))

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        train_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="training",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        validation_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="validation",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        history = model.fit(
          x = train_flow,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=validation_flow,
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=cpus_num,
          max_queue_size=128
          )
      if (test_results):
        test_flow = test_datagen.flow_from_directory(
            directory=test_dir,
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            x=test_flow,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(x_test)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(y_test, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [29]:
work_on_kmobilenet(show_model=True, run_fit=False, test_results=False)

Running: JP35B18-mobilenet-CIFAR10-26
Model: "kmobilenetv1-26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_35[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 32) 864         conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 32) 128         conv1[0][0]                      
______________________________________________

# Fitting

In [30]:
work_on_kmobilenet(show_model=False, run_fit=True, test_results=True)

Running: JP35B18-mobilenet-CIFAR10-26
Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Epoch 1/50
704/704 [==============================] - 438s 595ms/step - loss: 1.8526 - accuracy: 0.3194 - val_loss: 1.8724 - val_accuracy: 0.3438

Epoch 00001: val_accuracy improved from -inf to 0.34380, saving model to JP35B18-mobilenet-CIFAR10-26-best_result.hdf5
Epoch 2/50
704/704 [==============================] - 413s 585ms/step - loss: 1.4660 - accuracy: 0.4749 - val_loss: 4.2516 - val_accuracy: 0.2960

Epoch 00002: val_accuracy did not improve from 0.34380
Epoch 3/50
704/704 [==============================] - 415s 585ms/step - loss: 1.1940 - accuracy: 0.5793 - val_loss: 1.9151 - val_accuracy: 0.4984

Epoch 00003: val_accuracy improved from 0.34380 to 0.49840, saving model to JP35B18-mobilenet-CIFAR10-26-best_result.hdf5
Epoch 4/50
704/704 [==============================] - 414s 586ms/step - loss: 1.0140 - accuracy: 0.6454 - val_loss: 1.4859 - val_accuracy:

# Test Results

In [31]:
work_on_kmobilenet(show_model=False, run_fit=False, test_results=True)

Running: JP35B18-mobilenet-CIFAR10-26
Found 10000 images belonging to 10 classes.
Best Model Results: JP35B18-mobilenet-CIFAR10-26-best_result.hdf5
157/157 [==============================] - 18s 89ms/step - loss: 0.2525 - accuracy: 0.9302
loss 0.25251662731170654
acc 0.9301999807357788
Finished: JP35B18-mobilenet-CIFAR10-26
